In [151]:
from yummly import Client
import json
import requests
import pandas as pd
import numpy as np 
import re # need to import regex

In [44]:
# API call for the first 500 BB recipes labeled as such only!
header= {'X-Yummly-App-ID':'79663a75', 'X-Yummly-App-Key':'02b233108f476f3110e0f65437c4d6dd'}
url='http://api.yummly.com/v1/api/recipes?'
parameters={
            'allowedCourse[]':'course^course-Breakfast and Brunch',
            'excludedCourse[]': ['course^course-Main Dishes','course^course-Appetizers', 'course^course-Salads', 'course^course-Lunch',
                                'course^course-Side Dishes','course^course-Desserts','course^course-Breads',
                                 'course^course-Soups', 'course^course-Beverages', 'course^course-Condiments and Sauces',
                                'course^course-Cocktails', 'course^course-Snacks'],
            'maxResult': 501,
            'start': 499
            }

response=requests.get(url, headers = header, params = parameters)

In [45]:
response.status_code

200

In [146]:
BB=response.json()

print type(BB)
print BB.keys()

<type 'dict'>
[u'matches', u'totalMatchCount', u'attribution', u'facetCounts', u'criteria']


In [147]:
#only interrested in the information under matches. 
print len(BB['matches'])
print type(BB['matches'])
print BB['matches'][0].keys()

500
<type 'list'>
[u'flavors', u'rating', u'totalTimeInSeconds', u'ingredients', u'smallImageUrls', u'sourceDisplayName', u'recipeName', u'attributes', u'id', u'imageUrlsBySize']


In [117]:
#import previous list of recipes collected
df=pd.read_csv('BB_main.csv')
BB_ids=df.id
print BB_ids[0]
BB1_ids=[]
for recipe in BB_matches:
    BB1_ids.append(recipe['id'])
print BB1_ids[0]
#check if there are dupplicate recipes
[i for i, j in zip(BB_ids, BB1_ids) if i == j]

Raspberry-Melon-Breakfast-Bowls-1640967
Breakfast-Casserole-1549257


[]

In [148]:
#checkout one recipe
BB_matches=BB['matches']
BB_matches[0]

{u'attributes': {u'course': [u'Breakfast and Brunch']},
 u'flavors': None,
 u'id': u'Breakfast-Casserole-1549257',
 u'imageUrlsBySize': {u'90': u'https://lh3.googleusercontent.com/LKPCkmK59H7grKnOWmUzAwTxEUcRK3ksaQXbEqX1aqVJftFpmzgEKZlT8E1KuQAQ_rquuPHVMdWfjsMl07Dqrg=s90-c'},
 u'ingredients': [u'eggs',
  u'whole milk',
  u'croissants',
  u'bacon',
  u'jack cheese',
  u'salt',
  u'pepper'],
 u'rating': 4,
 u'recipeName': u'Breakfast Casserole',
 u'smallImageUrls': [u'https://lh3.googleusercontent.com/RTxRxF87IAxcCQ-tfGhjW-bnRvO0eNCd4XLKCLTl4WVbHA3aYj2cQvv6_2VpDIs3d4-qO4nfnKXvyzASIhnZBhY=s90'],
 u'sourceDisplayName': u'The Frugal Fairy',
 u'totalTimeInSeconds': 2700}

In [152]:
#forming lists to create dataframes of the features we want. 
main_list = []
ingredients_list = []
attributes_list = []
flavors_list = []

for food in BB_matches:

    _d1 = {}
    _d1['id'] = food['id']
    _d1['rating'] = food['rating']
    _d1['recipeName'] = food['recipeName']
    _d1['sourceDisplayName'] = food['sourceDisplayName']
    
    main_list.append(_d1)
    
    _d2 = {}
    _d2['id'] =food['id']
    for i in food['ingredients']:
        i = i.lower() # additional code to conver to lowercase
        i = re.sub(r'\d+%\s', '', i) # additional code to remove 1%, 2%, etc
        i = re.sub(r'\xae', '', i) # remove '\xae' characters
        _d2[i] = 1
    ingredients_list.append(_d2)

    _d3 = {}
    _d3['id'] = food['id']
    for k, v in food['attributes'].items():
        for i in v:
            _d3[i] = 1
    attributes_list.append(_d3)
    
    _d4 = {}
    _d4[food.get('id')]= food.get('flavors')
    flavors_list.append(_d4)

In [153]:
# read in dictionary for course and cuisine and create list of possible values for each
course_df = pd.read_csv('course_headers.csv', names=['course'])
cuisine_df = pd.read_csv('cuisine_headers.csv', names=['cuisine'])

course_list = course_df.course
cuisine_list= cuisine_df.cuisine


IOError: File course_headers.csv does not exist

In [58]:
#create dictionary of cuisine and course for each recipe
course_dict={}
cuisine_dict={}
for food in BB_matches:
    course_dict[food.get('id')]= food['attributes'].get('course')
    cuisine_dict[food.get('id')]= food['attributes'].get('cuisine')
    
_courses= {}

for k, v in course_dict.iteritems():
    course_val = {}
    for course in course_list:
        try:
            if course in v :
                course_val[course] = 1
            else:
                course_val[course] = 0
        except TypeError:
            course_val[course] = 0
    
        _courses[k] = course_val
        
_cuisines= {}       

for k, v in cuisine_dict.iteritems():
    cuisine_val = {}
    for course in cuisine_list:
        try:
            if course in v :
                cuisine_val[course] = 1
            else:
                cuisine_val[course] = 0
        except TypeError:
            cuisine_val[course] = 0
    
        _cuisines[k] = cuisine_val


In [59]:
#get list of recipe ids
recipe_ids=[]
for recipe in BB_matches:
    recipe_ids.append(recipe['id'])

In [60]:
# second api call to get other features for each recipe
key_id= '_app_id=79663a75&_app_key=02b233108f476f3110e0f65437c4d6dd'
url='http://api.yummly.com/v1/api/recipe/'

In [61]:
# retrieve other features for all recipes

def get_recipe(_id):
    response = requests.get(url + _id + '?' + key_id)
    return response.json()

recipes=[]
for _id in recipe_ids :
    recipes.append(get_recipe(_id))

In [62]:
response.status_code

200

In [63]:
print len(recipes)
print recipes[1].keys()

500
[u'totalTime', u'ingredientLines', u'attribution', u'name', u'prepTimeInSeconds', u'rating', u'numberOfServings', u'yield', u'nutritionEstimates', u'source', u'flavors', u'images', u'attributes', u'id', u'prepTime', u'totalTimeInSeconds']


In [64]:
#for each recipe create a new dictionary of selected attributes and append into a list

recipe_details=[]
for recipe in recipes:
    _dict={}
    #import pdb; pdb.set_trace()
    _dict['id']=recipe['id']
    _dict['ingredientCount']= len(recipe['ingredientLines'])
    _dict['numberOfServings']= recipe['numberOfServings']
    if 'prepTimeInSeconds' in recipe.keys():
        _dict['prepTimeInSeconds']= recipe['prepTimeInSeconds']
    else:
        _dict['prepTimeInSeconds']= None
    if 'cookTimeInSeconds' in recipe.keys():
        _dict['cookTimeInSeconds']= recipe['cookTimeInSeconds']
    else:
        _dict['cookTimeInSeconds']= None
    _dict['totalTimeInSeconds']=recipe['totalTimeInSeconds']
    
    
    recipe_details.append(_dict)
        

In [120]:
#create dataframes, arrange column index and save into csv
df_main = pd.DataFrame(main_list)
df_main.to_csv('BB_main_1.csv', encoding ='utf-8')

df_ingredients = pd.DataFrame(ingredients_list)
cols = list(df_ingredients)
cols.insert(0, cols.pop(cols.index('id')))
df_ingredients= df_ingredients.ix[:,cols]
df_ingredients.to_csv('BB_ingredients_1.csv', encoding ='utf-8')

df_attributes = pd.DataFrame(attributes_list)
cols = list(df_attributes)
cols.insert(0, cols.pop(cols.index('id')))
df_attributes = df_attributes.ix[:,cols]
df_attributes.to_csv('BB_attributes_1.csv')

df_flavors = pd.DataFrame(flavors_list).transpose()
df_flavors.reset_index(level=0, inplace=True)
df_flavors.to_csv('BB_flavors_1.csv')

df_courses = pd.DataFrame(_courses).transpose()
df_courses.reset_index(level=0, inplace=True)
df_courses.to_csv('BB_courses_1.csv')

df_cuisines = pd.DataFrame(_cuisines).transpose()
df_cuisines.reset_index(level=0, inplace=True)
df_cuisines.to_csv('BB_cuisines_1.csv')

df_details=pd.DataFrame(recipe_details)
cols = list(df_details)
cols.insert(0, cols.pop(cols.index('id')))
df_details=df_details.ix[:,cols]
df_details.to_csv('BB_details_1.csv')

In [121]:
df_ingredients

,id,1% low-fat milk,2% lowfat greek yogurt,2% reduced-fat milk,active dry yeast,agave nectar,all-purpose flour,allspice,almond butter,almond extract,...,whole wheat tortillas,whole wheat white flour,wild rice,worcestershire sauce,yeast,yellow onion,yellow peppers,yellow squash,yoghurt,zucchini
0,Breakfast-Casserole-1549257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Blueberry-Breakfast-Bake-1699614,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Broccoli_-ham_-and-mozzarella-baked-with-eggs-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cream-Cheese_-Blackberry_-and-Pistachio-Stuffe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Caramelized-Peach-Quinoa-Breakfast-Bowl-1684482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Cottage-Cheese-Pancakes-1639931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Quinoa-Breakfast-Bowl-1576732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cheesecake-Stuffed-French-Toast-Sliders-1691299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Egg-_-Nopalitos-Scramble--1683337,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Low-Carb-Gluten-Free-Musli-Cereal-1704968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df_ingredients.sum(axis=0).sort_values(ascending = False)

id                             Breakfast-Casserole-1549257Blueberry-Breakfast...
eggs                                                                         255
salt                                                                         223
milk                                                                         146
baking powder                                                                110
cinnamon                                                                     100
butter                                                                        83
vanilla extract                                                               79
sugar                                                                         75
large eggs                                                                    74
pepper                                                                        65
water                                                                         53
flour                       

In [82]:
x=['hello there','hello','there','bye']
x_=[]
for i in x:
    x_.append(i.split())
x_=[['hello', 'there'], ['hello'], ['there'], ['bye']]
for i in x:
    for y in i:
        z=['hello']
        if y in z:
            


[['hello', 'there'], ['hello'], ['there'], ['bye']]

In [78]:
for a, b in enumerate(x):
    print b

hello there
hello


In [79]:
df_ingredients.columns

Index([                                      u'id',
                                u'1% low-fat milk',
                         u'2% lowfat greek yogurt',
                            u'2% reduced-fat milk',
       u'Athenos Traditional Crumbled Feta Cheese',
                            u'Bisquick Baking Mix',
                          u'Country Crock® Spread',
                                u'English muffins',
                            u'French bread loaves',
                       u'Hormel Real Bacon Pieces',
       ...
                          u'whole wheat tortillas',
                        u'whole wheat white flour',
                                      u'wild rice',
                           u'worcestershire sauce',
                                          u'yeast',
                                   u'yellow onion',
                                 u'yellow peppers',
                                  u'yellow squash',
                                        u'yoghurt',
 

In [87]:
cols=list(df_ingredients)
unique_cols=[]
for i in df_ingredients.columns:
    for j in cols:
        if i not in j and i not in unique_cols:
            unique_cols.append(i)
            

In [122]:
df_ingredients.head()

,id,1% low-fat milk,2% lowfat greek yogurt,2% reduced-fat milk,active dry yeast,agave nectar,all-purpose flour,allspice,almond butter,almond extract,...,whole wheat tortillas,whole wheat white flour,wild rice,worcestershire sauce,yeast,yellow onion,yellow peppers,yellow squash,yoghurt,zucchini
0,Breakfast-Casserole-1549257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Blueberry-Breakfast-Bake-1699614,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Broccoli_-ham_-and-mozzarella-baked-with-eggs-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cream-Cheese_-Blackberry_-and-Pistachio-Stuffe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Caramelized-Peach-Quinoa-Breakfast-Bowl-1684482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
BB = pd.read_csv('BB_ingredients.csv')

In [124]:
BB.head()

,Unnamed: 0,id,2% reduced-fat milk,Chobani Yogurt,Crunch Cereal,English muffins,Himalayan salt,Honeysuckle White® Turkey Breakfast Sausage Patties,Jarlsberg,KRAFT Shredded Pepper Jack Cheese with a TOUCH OF PHILADELPHIA,...,whole wheat flour,whole wheat pastry flour,whole wheat tortillas,whole wheat white flour,wild rice,yellow cornmeal,yellow onion,yellow squash,yoghurt,zucchini
0,0,Raspberry-Melon-Breakfast-Bowls-1640967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Baked-Egg-In-Avocado-1686638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,The-28-Day-Shrink-Your-Stomach-Challenge-Break...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Vanilla-Peach-Overnight-Oats-1706871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Old-Fashioned-Pancakes-1639238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
data=pd.concat([BB, df_ingredients],join='outer', axis=0, ignore_index=True)


//anaconda/lib/python2.7/site-packages/pandas/indexes/base.py:2028: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  indexer = self._engine.get_indexer(target._values)


In [104]:
type(df_ingredients)

pandas.core.frame.DataFrame

In [102]:
data

//anaconda/lib/python2.7/site-packages/pandas/indexes/base.py:1229: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return key in self._engine


,2% reduced-fat milk,Chobani Yogurt,Crunch Cereal,English muffins,Himalayan salt,Honeysuckle White® Turkey Breakfast Sausage Patties,Jarlsberg,KRAFT Shredded Pepper Jack Cheese with a TOUCH OF PHILADELPHIA,KRAFT Shredded Sharp Cheddar Cheese,Kraft Grated Parmesan Cheese,...,whole wheat white flour,wild rice,worcestershire sauce,yeast,yellow cornmeal,yellow onion,yellow peppers,yellow squash,yoghurt,zucchini
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0


In [111]:
for i in df_ingredients.columns:
    if 'English' in i:
        print i

English muffins
vegan English muffins
